In [6]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [97]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_formula.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [7]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [99]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [100]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [101]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [102]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [104]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[2.6108e+02, 1.1000e+00],
         [5.4318e+01, 7.0420e-05],
         [5.4410e+01, 7.0990e-05],
         ...,
         [2.5980e+02, 7.6210e-05],
         [2.6686e+02, 7.2440e-05],
         [2.7990e+02, 7.6270e-05]],

        [[4.5916e+02, 1.1000e+00],
         [6.3764e+01, 1.6300e-03],
         [6.3768e+01, 2.1400e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.5916e+02, 1.1000e+00],
         [5.5018e+01, 8.8407e-03],
         [6.7054e+01, 1.8401e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[5.4313e+02, 1.1000e+00],
         [1.1506e+02, 7.3058e-04],
         [1.2651e+02, 3.6875e-04],
         ...,
         [2.2003e+02, 1.2288e-03],
         [3.4506e+02, 6.1333e-03],
         [3.4607e+02, 8.9567e-04]],

        [[1.5911e+02, 1.1000e+00],
         [1.0306e+02, 7.

In [105]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256,  68, 256, 256, 256, 256, 256, 256])


In [106]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [107]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [108]:
tmp_original['precursor_mz']

tensor([261.0757, 459.1551, 359.1635, 163.0400, 282.0907, 733.3400, 445.2116,
        419.2289, 314.1400, 543.1280, 159.1100, 284.1412])

In [109]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+'],)

In [110]:
tmp_original['mol_freq']

tensor([ 74.,   3.,  43., 119.,  13.,  46.,  18.,   3.,  59.,   5., 158.,  90.])

In [111]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [112]:
sum(tmp_original['labels'])

tensor(12)

In [113]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1348, 1604, 1860, 2116, 2372, 2628])

In [114]:
len(tmp_original['labels'])

2884

In [115]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256, 256,  68, 256, 256, 256, 256, 256, 256])

In [116]:
sum(tmp_original['batch_ptr'])

tensor(2884)

In [117]:
len(tmp_original['candidates'])

2884

In [118]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [119]:
len(tmp_original['candidates_smiles'])

2884

# MSnRetrieval

In [9]:
from massspecgym.featurize import SpectrumFeaturizer

In [10]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_formula.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [11]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [12]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [13]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 15674
Dataset length: 15674


In [14]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [15]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 11883
Val dataset size: 1899


In [16]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

in collate_fn
{'spec': DataBatch(x=[137, 1039], edge_index=[2, 125], batch=[137], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([477.2384, 356.1493, 371.1853, 326.0504, 384.1442, 628.2236, 445.2122,
        427.3027, 381.2060, 465.2748, 429.1057, 804.2921]), 'adduct': ['[M+H]+', '[M+H]+', '[M+H]+', '[M+Na]+', '[M+H]+', '[M+NH4]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+NH4]+'], 'identifier': ['0093945_0000000', '0048361_0000000', '0000205_0000000', '0041577_0000000', '0091010_0000000', '0002522_0000000', '0048098_0000000', '0077795_0000000', '0074678_0000000', '0017581_0000000', '0035304_0000000', '0008924_0000000'], 'mol_freq': tensor([1., 1., 1., 2., 1., 3., 1., 1., 1., 1., 1., 1.]), 'smiles': ['CN([C@@H]1CC[C@]2([C@H]3CC4=

In [17]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256,  70, 256, 256, 138,  95])


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [129]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [130]:
tmp_msn['spec'] 

DataBatch(x=[151, 1039], edge_index=[2, 139], batch=[151], ptr=[13])

In [131]:
tmp_msn['precursor_mz']

tensor([ 375.1663,  581.1501,  355.1513,  583.1467,  354.1772,  379.1365,
         234.1965,  272.1281,  363.1064,  367.1401,  397.1547, 1154.6317])

In [132]:
tmp_msn['mol_freq']

tensor([1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2.])

In [133]:
tmp_msn['identifier']

['0048452_0000000',
 '0007538_0000000',
 '0050132_0000000',
 '0099539_0000000',
 '0051053_0000000',
 '0027843_0000000',
 '0083360_0000000',
 '0078665_0000000',
 '0064692_0000000',
 '0045890_0000000',
 '0039807_0000000',
 '0000746_0000000']

In [134]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+']

In [135]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [136]:
tmp_msn['smiles']

['CC(=O)C1=CN(C2=CC=CC=C21)CC(=O)N[C@H](CCCNC(=O)N)C(=O)O',
 'C1[C@@]([C@H]([C@@H](O1)OC[C@@H]2[C@H]([C@@H]([C@H]([C@@H](O2)OC3=CC(=CC4=C3C(=CC(=O)O4)C5=CC(=C(C=C5)O)O)O)O)O)O)O)(CO)O',
 'COCCNC1=NC(=NC2=NC=CN=C21)NCC3=CC4=C(C=C3)OCO4',
 'C1CC(=O)N([C@@H]1C(=O)N(C2=CC(=CN=C2)F)[C@H](C3=CC=CC=C3Cl)C(=O)NC4CC(C4)(F)F)C5=NC=CC(=C5)C#N',
 'CC(CNC1=NC2=C(N1CC(COCC=C)O)C(=O)NC(=O)N2C)O',
 'C1C(CC(=O)C2=C1NC3=NC=NN3C2C4=CC(=CC(=C4)F)F)C5=CC=CC=C5',
 'CCCCCCNC(=NCC1=CC=CC=C1)N',
 'C1CNC(C2=CC(=C(C=C21)O)O)CC3=CC=C(C=C3)O',
 'COC1=CC2=NC=CC(=C2C=C1)NC(=O)NC3=CC=CC(=N3)C(F)(F)F',
 'C[C@@H](C(=O)N[C@@H](CC1=CC=CC=C1)C(=O)O)N2C(=O)C3=CC=CC=C3N=N2',
 'C1CC2=C3C(=CC=C2)C(=C(C(=O)N3C1)C(=O)NC4=CC=CC=C4C5=CC=CC=C5)O',
 'CC1C(C(CC(O1)O[C@H]2CC[C@@]3(C4[C@@H]([C@H]([C@@]5([C@H](CC[C@@]5([C@@]4(CC=C3C2)O)O)[C@H](C)O)C)O)O)C)OC)OC6CC(C(C(O6)C)OC7CC(C(C(O7)C)OC8C(C(C(C(O8)C)OC9C(C(C(C(O9)CO)O)O)O)OC)O)OC)OC']

In [137]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [138]:
sum(tmp_msn['labels'])

tensor(12)

In [139]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768,  831, 1087, 1343, 1599, 1855, 2111, 2367, 2623])

In [140]:
len(tmp_msn['labels'])

2647

In [141]:
tmp_msn['batch_ptr']

tensor([256, 256, 256,  63, 256, 256, 256, 256, 256, 256, 256,  24])

In [142]:
sum(tmp_msn['batch_ptr'])

tensor(2647)

In [143]:
len(tmp_msn['candidates'])

2647

In [144]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [145]:
tmp_msn.keys() == tmp_original.keys()

True

In [146]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [147]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [148]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

True

In [149]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2647), 2647)

In [150]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(2884), 2884)

In [151]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [152]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [153]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [154]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [155]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [156]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [157]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [158]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [159]:
len(tmp_msn['candidates_smiles'])

2647